In [6]:
import tensorflow as tf
from tensorflow import keras

from sklearn.preprocessing import OneHotEncoder

from keras.layers import Input, Dense, Activation, Dropout
from keras.models import Sequential
from keras.models import Model

import numpy as np
import matplotlib.pyplot as plt
from keras.optimizers import Adam
import pandas as pd

print(tf.__version__)

#the first thing we need to do is get the data and the labels for the data. 
DataFrameMain = pd.read_csv("C:/Users/tsmar/election_dataframe.csv")
DataFrame = DataFrameMain.select_dtypes(include=["int", "float"])
headers = list(DataFrame.columns.values)

jj = int(0.95*len(DataFrame))
kk = int(0.05*len(DataFrame))

#these are the columns I will train on for now.
DataFrame["meanGOP"] = DataFrame[["Republicans 2016", "Republicans 2012", "Republicans 2008"]].mean(axis=1) / 100
DataFrame["meanDEM"] = DataFrame[["Democrats 2016", "Democrats 2012", "Democrats 2008"]].mean(axis=1) / 100
DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] = DataFrame["Percent of adults with a bachelor's degree or higher, 2012-2016"] / 100
DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] = DataFrame["Percent of adults completing some college or associate's degree, 2012-2016"] / 100
DataFrame["Percent of adults with a high school diploma only, 2012-2016"] = DataFrame["Percent of adults with a high school diploma only, 2012-2016"] / 100
DataFrame["Percent of adults with less than a high school diploma, 2012-2016"] = DataFrame["Percent of adults with less than a high school diploma, 2012-2016"]/100
#make test and train data split 80/20
trainData = DataFrame[["meanGOP","meanDEM","Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]][0:jj]
testData = DataFrame[["meanGOP","meanDEM","Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]][-1*kk:]
trainLabels = []
testLabels = []
#build the train Labels
for index in range(0, len(trainData["meanGOP"])):
    rawData = trainData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        trainLabels.append(np.array([1, 0])) #gop
    else:
        trainLabels.append(np.array([0, 1])) #dem
    
#build the test labels
for index in range(0, len(testData["meanGOP"])):
    rawData = testData.values
    j = rawData[index][0]
    k = rawData[index][1]
    if j > k:
        testLabels.append(np.array([1, 0])) #gop
    else:
        testLabels.append(np.array([0, 1])) #dem
        
data_train = np.array(trainData[["Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]].values, dtype='float64')
data_test = np.array(testData[["Percent of adults with a bachelor's degree or higher, 2012-2016","Percent of adults completing some college or associate's degree, 2012-2016","Percent of adults with a high school diploma only, 2012-2016","Percent of adults with less than a high school diploma, 2012-2016"]].values, dtype='float64') 

model = Sequential()
model.add(Dense(64, input_dim=4, activation='relu', init='uniform'))

model.add(Dense(64, activation='relu',init='uniform'))

model.add(Dropout(rate=0.5, ))

model.add(Dense(64, activation='relu',init='uniform'))

model.add(Dropout(rate=0.125, ))

model.add(Dense(64, activation='relu',init='uniform'))

model.add(Dense(2, activation='softmax', init="uniform"))

model.compile(optimizer=Adam(lr=0.007),loss='categorical_crossentropy',metrics=['accuracy'])

model.fit(data_train, np.array(trainLabels), epochs=3, verbose=1)

scores = model.evaluate(data_test, np.array(testLabels))
print("\n%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
#for j in range(len(data_train)):
    #print(data_train[j], " should get mapped to: ", trainLabels[j])

1.11.0


C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\tsmar\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexi

Epoch 1/3
2961/2961 [==============================] - 1s 349us/step - loss: nan - acc: 0.7893
Epoch 2/3
2961/2961 [==============================] - 0s 62us/step - loss: nan - acc: 0.7960
Epoch 3/3
155/155 [==============================] - 0s 1ms/step

acc: 80.65%
